<img src='justtemp (3).png' width='75%'>

In [111]:
import numpy as np

In [112]:
num_samples = 10

image_height = 5
image_width = 5

positions = [
    (0, 0),  
    (0, 3),  
    (3, 0),  
    (3, 3)   
]

x = np.zeros((num_samples, 1,image_height, image_width))
y = np.zeros((num_samples))

for i in range(num_samples):
    pos = positions[i % len(positions)]
    x[i,0, pos[0]:pos[0]+2, pos[1]:pos[1]+2] = 1
    
    y[i] = 1 if pos[0] == 0 else 0

x[0], y[0]

(array([[[1., 1., 0., 0., 0.],
         [1., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]]),
 1.0)

In [113]:
x =x.reshape(x.shape[0], 1, 5, 5)
print('num_img, channel, height, width')
x.shape

num_img, channel, height, width


(10, 1, 5, 5)

In [114]:
print('num_img')
y.shape

num_img


(10,)

In [115]:
weights = np.random.randn(1, 3, 3)

bias = np.random.randn(1)

print(weights)
print(bias)

[[[ 0.36180609  0.93718509  0.51518255]
  [-0.4477891  -0.63154557  0.57045719]
  [ 0.06009869  1.16044147 -0.27702692]]]
[0.22662933]


$out = \cfrac{in - filter + 2padding}{stride} + 1$

In [116]:
def conv(x, weights, bias, padding=0):

    num_filters = weights.shape[0]

    out = x.shape[2] - weights.shape[2] + 2 * padding + 1
    z = np.zeros((x.shape[0], num_filters, out, out))

    x= np.pad(x, ((0, 0), (0, 0), (padding, padding), (padding, padding)), mode='constant', constant_values=0)

    for n in range(x.shape[0]):

        for r in range(num_filters):
            for k in range(x.shape[1]):

                for h in range(x.shape[2]):

                    if h+weights.shape[2] > x.shape[2]:
                        continue
                    
                    for w in range(x.shape[3]):

                        if w+weights.shape[2] > x.shape[3]:
                            continue

                        x_crop = x[n, k, h:h+weights.shape[2], w:w+weights.shape[2]]

                        z[n, r, h, w] += np.sum(x_crop * weights[r, k]) + bias[r]

    return z

```
Input image
    ||
    \/
Conv(3,3) x1
    ||
    \/
Full conected
```

In [117]:
def fc(x, weights, bias):
    z = np.dot(weights,x.T) + bias
    return z

In [118]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def relu(x):
    return np.maximum(0, x)

def binary_cross_entripy(y_hat, y):
    y_hat = np.clip(y_hat, 1e-8, 1 - 1e-8)
    return 1/y.shape[0] * np.sum(-(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat)))

In [119]:
weights1 = np.random.randn(5,x.shape[1], 3, 3)
bias1 = np.random.randn(5)
weights11 = np.random.randn(5,5, 2, 2)
bias11 = np.random.randn(5)
weights2 = np.random.randn(1, weights11.shape[0] * weights11.shape[2] * weights11.shape[3])
bias2 = np.random.randn(1)

z1 = conv(x, weights1, bias1)
a1 = relu(z1)

z11 = conv(a1, weights11, bias11)
a11 = relu(z11)

a11 = a11.reshape(a11.shape[0], -1)

z2 = fc(a11, weights2, bias2)
a2 = sigmoid(z2)

In [120]:
a2

array([[0.99999817, 0.99999961, 1.        , 1.        , 0.99999817,
        0.99999961, 1.        , 1.        , 0.99999817, 0.99999961]])

In [121]:
binary_cross_entripy(a2, y)

7.36827295965319

In [122]:
def conv_backprop(dz, weight):

    out = dz.shape[2] - weight.shape[2] + 2*1 + 1

    dinput = np.zeros((dz.shape[0], weight.shape[0], out, out))

    flipped_weight = np.flip(weight, axis=(1,2))
    
    dzm = dz.reshape(1, dz.shape[0], dz.shape[1], dz.shape[2])
    fwm = flipped_weight.reshape(1, flipped_weight.shape[0], flipped_weight.shape[1], flipped_weight.shape[2])
    
    res = conv(dzm, fwm, np.zeros((weight.shape[0])), padding=1)

    # print(res.shape) 1 5 3 3

    return res

In [123]:
da2 = - y.T/a2 + (1 - y.T)/(1 - a2)
dz2 = a2 * (1 - a2) * da2
dweights2 = 1/y.shape[0] * np.dot(dz2, a11)
dbias2 = 1/y.shape[0] * np.sum(dz2)

 
da11 = np.dot(dz2.T,weights2)
dz11 = da11 * np.where(a11 > 0, 1, 0)
dz11 = dz11.reshape(z11.shape)

dweights11 = np.zeros_like(weights11)
dbias11 = np.zeros_like(bias11)

for i in range(weights11.shape[0]):
    for j in range(weights11.shape[1]):
        dweights11[i, j] = np.mean(conv(a1[:, j].reshape(a1.shape[0], 1, a1.shape[2], a1.shape[3]), np.mean(dz11, axis=0)[i].reshape(1, 1, dz11.shape[2], dz11.shape[3]), np.ones_like(bias11)), axis=(0,1))
    dbias11[i] = np.sum(dz11[:, i])


da1 = np.zeros_like(a1)
for j in range(da1.shape[0]):
    for i in range(weights11.shape[0]): 
        da1[j, i] += conv_backprop(dz11[j], weights11[i])[0,0]

dz1 = da1 * np.where(a1 > 0, 1, 0)
dz1 = dz1.reshape(z1.shape)

dweights1 = np.zeros_like(weights1)
dbias1 = np.zeros_like(bias1)

for i in range(weights1.shape[0]):
    for j in range(weights1.shape[1]):
        dweights1[i, j] = np.mean(conv(x[:, j].reshape(x.shape[0], 1, x.shape[2], x.shape[3]), np.mean(dz1, axis=0)[i].reshape(1, 1, dz1.shape[2], dz1.shape[3]), np.ones_like(bias1)), axis=(0,1))
    dbias1[i] = np.sum(dz1[:, i])


learning_rate = 0.001
weights2 -= learning_rate * dweights2
bias2 -= learning_rate * dbias2

weights11 -= learning_rate * dweights11
bias11 -= learning_rate * dbias11

weights1 -= learning_rate * dweights1
bias1 -= learning_rate * dbias1

In [129]:
weights1 = np.random.randn(8,x.shape[1], 3, 3)
bias1 = np.random.randn(8)
weights11 = np.random.randn(8,8, 2, 2)
bias11 = np.random.randn(8)
weights2 = np.random.randn(1, weights11.shape[0] * weights11.shape[2] * weights11.shape[3])
bias2 = np.random.randn(1)

for i in range(3000): 

    z1 = conv(x, weights1, bias1)
    a1 = relu(z1)

    z11 = conv(a1, weights11, bias11)
    a11 = relu(z11)

    a11 = a11.reshape(a11.shape[0], -1)

    z2 = fc(a11, weights2, bias2)
    a2 = sigmoid(z2)


    if i % 1000 == 0:
        print(binary_cross_entripy(a2, y))

    learning_rate = 0.1


    da2 = - y.T/a2 + (1 - y.T)/(1 - a2 + 1e-8)
    dz2 = a2 * (1 - a2) * da2
    dweights2 = 1/y.shape[0] * np.dot(dz2, a11)
    dbias2 = 1/y.shape[0] * np.sum(dz2)

    
    da11 = np.dot(dz2.T,weights2)
    dz11 = da11 * np.where(a11 > 0, 1, 0)
    dz11 = dz11.reshape(z11.shape)

    dweights11 = np.zeros_like(weights11)
    dbias11 = np.zeros_like(bias11)

    for i in range(weights11.shape[0]):
        for j in range(weights11.shape[1]):
            dweights11[i, j] = np.mean(conv(a1[:, j].reshape(a1.shape[0], 1, a1.shape[2], a1.shape[3]), np.mean(dz11, axis=0)[i].reshape(1, 1, dz11.shape[2], dz11.shape[3]), np.ones_like(bias11)), axis=(0,1))
        dbias11[i] = np.sum(dz11[:, i])


    da1 = np.zeros_like(a1)
    for j in range(da1.shape[0]):
        for i in range(weights11.shape[0]): 
            da1[j, i] += conv_backprop(dz11[j], weights11[i])[0,0]

    dz1 = da1 * np.where(a1 > 0, 1, 0)
    dz1 = dz1.reshape(z1.shape)

    dweights1 = np.zeros_like(weights1)
    dbias1 = np.zeros_like(bias1)

    for i in range(weights1.shape[0]):
        for j in range(weights1.shape[1]):
            dweights1[i, j] = np.mean(conv(x[:, j].reshape(x.shape[0], 1, x.shape[2], x.shape[3]), np.mean(dz1, axis=0)[i].reshape(1, 1, dz1.shape[2], dz1.shape[3]), np.ones_like(bias1)), axis=(0,1))
        dbias1[i] = np.sum(dz1[:, i])
 

    weights2 -= learning_rate * dweights2
    bias2 -= learning_rate * dbias2

    weights11 -= learning_rate * dweights11
    bias11 -= learning_rate * dbias11

    weights1 -= learning_rate * dweights1
    bias1 -= learning_rate * dbias1

5.836239001242696
0.0001398882011188859
5.4381677186775974e-05


In [130]:
np.round(a2)

array([[1., 1., 0., 0., 1., 1., 0., 0., 1., 1.]])

In [131]:
y

array([1., 1., 0., 0., 1., 1., 0., 0., 1., 1.])